Todo 
- calcualte industry-household for country selection (already defined in first cell)
- calculate European demand for each of the indicators, return which countries are included in each calculation

In [96]:
# Import necessary libraries
import pandas as pd
from src.utils.filter_conditions_monthly import filter_conditions_monthly
from src.utils.functions import (
    calculate_industry_demand_from_industry_power,
    calculate_totals_for_countries,
    calculate_totals_monthly,
    calculate_industry_from_power_monthly
)

# Define constants
CALCULATE_INDUSTRY_DEMAND_COUNTRIES = ['HU', 'LU', 'PT', 'RO']
CALCULATE_COUNTRY_TOTALS = ['BE', 'FR', 'HU', 'IT', 'LU', 'NL', 'PT', 'RO']
CALCULATE_INDUSTRY_POWER_COUNTRIES = ['AT', 'BG', 'HR', 'CZ', 'DK', 'EE', 'ES', 'FI', 'GR', 'LV', 'LT', 'PL', 'SK', 'SI', 'SE']

# define paths 
EUROSTAT_PATH = "src/data/raw/eurostat/latest_data.csv"
BNETZA_PATH = "src/data/raw/germany_household/latest_data.csv"


In [97]:
# 1. Load and preprocess daily demand data
df = pd.read_csv("src/data/processed/daily_demand_all.csv")
df['date'] = pd.to_datetime(df['date'].astype(str).str[:10])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Group by relevant columns and convert to TWh
df = df.groupby(['country', 'type', 'source', 'year', 'month'], as_index=False).agg({'demand': 'sum'})
df['demand'] = df['demand'] / 1000000000

# Display first few rows
print("Daily demand data after initial processing:")
display(df.head())

Daily demand data after initial processing:


,country,type,source,year,month,demand
0,AT,power,energy-charts,2018,12,0.002566
1,AT,power,energy-charts,2019,1,3.147638
2,AT,power,energy-charts,2019,2,2.488796
3,AT,power,energy-charts,2019,3,1.331403
4,AT,power,energy-charts,2019,4,1.055020


In [98]:
# 2. Load and process Eurostat data
eurostat_df = pd.read_csv(EUROSTAT_PATH)
eurostat_df['date'] = pd.to_datetime(eurostat_df['date'])
eurostat_df['month'] = eurostat_df['date'].dt.month
eurostat_df['year'] = eurostat_df['date'].dt.year
del eurostat_df['date']
df = pd.concat([df, eurostat_df], ignore_index=True)

print("\nAfter adding Eurostat data:")
display(df[df['source'] == 'eurostat'].head())


After adding Eurostat data:


,country,type,source,year,month,demand
5050,AT,total,eurostat,2016,1,12.491121
5051,BE,total,eurostat,2016,1,21.026683
5052,BG,total,eurostat,2016,1,4.403337
5053,CY,total,eurostat,2016,1,0.000000
5054,CZ,total,eurostat,2016,1,12.906955


In [99]:
# 3. Load and process BNetzA data 
bnetza_df = pd.read_csv(BNETZA_PATH)
bnetza_df['date'] = pd.to_datetime(bnetza_df['date'])
bnetza_df['month'] = bnetza_df['date'].dt.month
bnetza_df['year'] = bnetza_df['date'].dt.year
bnetza_append = bnetza_df[['country', 'type', 'source', 'year', 'month', 'demand']]
df = pd.concat([df, bnetza_append], ignore_index=True)

print("\nAfter adding BNetzA data:")
display(df[(df['source'] == 'bundesnetzagentur') & (df['year'] == 2025)])


After adding BNetzA data:


,country,type,source,year,month,demand
8186,DE,household,bundesnetzagentur,2025,1,61.783


In [100]:
# 4. Apply filtering conditions
conditions_df = pd.DataFrame(filter_conditions_monthly, columns=['country', 'type', 'source'])
filtered_df = df.merge(conditions_df, on=['country', 'type', 'source'])

print("\nAfter applying filters:")
display(filtered_df.head())


After applying filters:


,country,type,source,year,month,demand
0,AT,power,energy-charts,2018,12,0.002566
1,AT,power,energy-charts,2019,1,3.147638
2,AT,power,energy-charts,2019,2,2.488796
3,AT,power,energy-charts,2019,3,1.331403
4,AT,power,energy-charts,2019,4,1.055020


In [101]:
# 5. Aggregate data
aggregated_df = (
    filtered_df.groupby(['country', 'type', 'year', 'month'], as_index=False)
    .agg({
        'demand': 'sum',
        'source': lambda x: ', '.join(sorted(set(x))) if len(set(x)) > 1 else x.iloc[0]
    })
)

print("\nAggregated data:")
display(aggregated_df.head())


Aggregated data:


,country,type,year,month,demand,source
0,AT,power,2018,12,0.002566,energy-charts
1,AT,power,2019,1,3.147638,energy-charts
2,AT,power,2019,2,2.488796,energy-charts
3,AT,power,2019,3,1.331403,energy-charts
4,AT,power,2019,4,1.055020,energy-charts


In [102]:
# 6. Calculate industry demand for specific countries
industry_df = calculate_industry_from_power_monthly(aggregated_df, CALCULATE_INDUSTRY_DEMAND_COUNTRIES)
updated_df = pd.concat([aggregated_df, industry_df], ignore_index=True)
updated_df = updated_df.drop_duplicates()
updated_df = updated_df.sort_values(by=['country', 'type', 'year', 'month']).reset_index(drop=True)

print("\nAfter calculating industry demand:")
display(updated_df.head())

Filtering data for countries: ['HU', 'LU', 'PT', 'RO']
Filtered DataFrame shape: (588, 6)
Pivoted DataFrame shape: (296, 5)
Final industry DataFrame shape: (292, 6)

After calculating industry demand:


,country,type,year,month,demand,source
0,AT,power,2018,12,0.002566,energy-charts
1,AT,power,2019,1,3.147638,energy-charts
2,AT,power,2019,2,2.488796,energy-charts
3,AT,power,2019,3,1.331403,energy-charts
4,AT,power,2019,4,1.055020,energy-charts


In [103]:
# 7. Calculate country totals
updated_df = calculate_totals_monthly(updated_df, CALCULATE_COUNTRY_TOTALS)

print("\nAfter calculating country totals:")
display(updated_df.head())


After calculating country totals:


,country,type,year,month,demand,source
0,AT,power,2018,12,0.002566,energy-charts
1,AT,power,2019,1,3.147638,energy-charts
2,AT,power,2019,2,2.488796,energy-charts
3,AT,power,2019,3,1.331403,energy-charts
4,AT,power,2019,4,1.055020,energy-charts


In [104]:
# 8. Calculate German industry demand
german_filter_df = updated_df[updated_df['country'] == 'DE']
german_pivot_df = german_filter_df.pivot_table(
    index=['country', 'year', 'month'], 
    columns='type', 
    values='demand',
    aggfunc='sum'
).reset_index()

german_pivot_df['industry_demand'] = german_pivot_df['total'] - german_pivot_df['household'] - german_pivot_df['power']
german_industry_df = german_pivot_df[['country', 'year', 'month', 'industry_demand']].dropna(subset=['industry_demand'])
german_industry_df = german_industry_df.rename(columns={'industry_demand': 'demand'})
german_industry_df['type'] = 'industry'
german_industry_df['source'] = 'calculated'

updated_df = pd.concat([updated_df, german_industry_df], ignore_index=True)

print("\nAfter calculating German industry demand:")
display(updated_df[updated_df['country'] == 'DE'].head())



After calculating German industry demand:


,country,type,year,month,demand,source
769,DE,household,2018,1,59.210,bundesnetzagentur
770,DE,household,2018,2,68.348,bundesnetzagentur
771,DE,household,2018,3,60.481,bundesnetzagentur
772,DE,household,2018,4,21.960,bundesnetzagentur
773,DE,household,2018,5,11.594,bundesnetzagentur


In [105]:
# 9. Final processing and export
final_df = updated_df[updated_df['year'] >= 2019].copy()
final_df['demand'] = final_df['demand'].round(2)

print("\nFinal dataset sample:")
display(final_df.head())


Final dataset sample:


,country,type,year,month,demand,source
1,AT,power,2019,1,3.15,energy-charts
2,AT,power,2019,2,2.49,energy-charts
3,AT,power,2019,3,1.33,energy-charts
4,AT,power,2019,4,1.06,energy-charts
5,AT,power,2019,5,0.16,energy-charts


In [106]:
# drop all values where demand is 0 or missing
print(len(final_df))
final_df = final_df[final_df['demand'] != 0]
final_df = final_df[final_df['demand'].notna()]
print(len(final_df))

#drop all the industry_power rows not to double count
final_df = final_df[~(final_df['country'].isin(CALCULATE_INDUSTRY_DEMAND_COUNTRIES) & (final_df['type'] == 'industry-power'))]

5288
5157


In [107]:
# Save the results
final_df.to_csv("src/data/analyzed/monthly_demand_clean.csv", index=False)